# Decision Analysis - Project 1 #


Mateusz Małecki <br>
Daniel Jankowski 148257

In [13]:
import pandas as pd
import numpy as np
import math
from typing import Optional


In [2]:
data = pd.read_csv("DA_database.csv", decimal=",")

In [3]:
data.head()

,name,prestige,power,price,engine_size,colour_preference,fuel_consumption,mileage,production_year,automatic_gear_box
0,Seat Ibiza 1.2 white,1,90,34900,1.2,2,6.0,90000,2015,0
1,Seat Ibiza 1.4 green,1,85,27800,1.4,4,6.9,82000,2015,0
2,Skoda Fabia grey,4,75,31500,1.0,1,5.8,215000,2019,0
3,Nissan Note grey,3,80,28500,1.2,1,6.0,133000,2014,0
4,MINI Cooper 1.6 blue,7,175,27900,1.6,5,2.0,183000,2006,0


## Decision classes ##

In [4]:
params={
    "prestige":{
    "type":"gain", "p":2, "q":1, "v":None, "w":5
    },
    "power":{
    "type":"gain", "p":15, "q":5, "v":None, "w":7
    },
    "price":{
    "type":"cost", "p":3000, "q":1000, "v":None, "w":8
    },
    "engine_size":{
    "type":"cost", "p":0.2, "q":0, "v":None, "w":7
    },
    "colour_preference":{
    "type":"gain", "p":0, "q":0, "v":None, "w":2
    },
    "fuel_consumption":{
    "type":"cost", "p":0.3, "q":0.1, "v":None, "w":8
    },
    "mileage":{
    "type":"cost", "p":35000, "q":20000, "v":None, "w":10
    },
    "production_year":{
    "type": "gain", "p":4, "q":2, "v":None, "w":6
    },
    "automatic_gear_box":{
    "type":"gain", "p":1, "q":0, "v":None, "w":3
    },
}

## Promethe ##

In [66]:
for x in zip(range(10), data["name"]):
    print(x)

(0, 'Seat Ibiza 1.2 white')
(1, 'Seat Ibiza 1.4 green')
(2, 'Skoda Fabia grey')
(3, 'Nissan Note grey')
(4, 'MINI Cooper 1.6 blue')
(5, 'Opel Corsa white')
(6, 'Toyota Yaris 1.3 grey')
(7, 'Citroen DS3 red')
(8, 'Volkswagen Polo 1.2 white')
(9, 'Toyota Yaris 1.0 white')


In [71]:
class Promethee:
    def __init__(self, alternatives: pd.DataFrame, parameters: dict[str, dict]) -> None:
        self.alternatives = alternatives
        self.parameters = parameters
        self.criteria = [x for x in alternatives.columns[1:]]


    def criteriaComparison(self, criteria: str, a, b, funcType: str) -> float:
        assert(funcType in ["Type1", "Type2", "Type3", "Type4", "Type5", "Type6"])
        p, q = self.parameters[criteria]["p"], self.parameters[criteria]["q"]
        diff = a - b if self.parameters[criteria]["type"] == "gain" else b-a
        if funcType == "Type1":
            return 1 if diff > 0 else 0
        
        elif funcType == "Type2":
            return 1 if diff > q else 0
        
        elif funcType == "Type3":
            if diff > p:
                return 1
            elif diff <= 0:
                return 0
            else:
                return diff / p
        
        elif funcType == "Type4":
            if diff > p:
                return 1
            elif diff < q:
                return 0
            else:
                return 0.5
        
        elif funcType == "Type5":
            if diff > p:
                return 1
            elif diff <= q:
                return 0
            else:
                return (diff - q)/(p-q)
        else:
            return 0 if diff <= 0 else 1 - math.exp(-diff/2) # assuming sd = 1
    
    def computeMarginalPreferencesAllCriteria(self, funcType: str) -> np.array:
        marginalPreferenceIndices = np.zeros((len(self.criteria), len(self.alternatives), len(self.alternatives)))
        for n, criteria in enumerate(self.criteria):
            for i, row in self.alternatives.iterrows():
                for j, row2 in self.alternatives.iterrows():
                    if i == j:
                        continue
                    marginalPreferenceIndices[n][i][j] = self.criteriaComparison(criteria, row[criteria], row2[criteria], funcType)
        return marginalPreferenceIndices
    
    def computeComprehensiveMatrix(self, funcType: str) -> np.array:
        print(np.array([x['w'] for x in self.parameters.values()]))
        return np.sum(self.computeMarginalPreferencesAllCriteria(funcType) * np.array([x['w'] for x in self.parameters.values()]), axis=0)
    
    def computePositiveFlows(self, matrix) -> np.array:
        return np.sum(matrix, axis = 1)
    def computeNegativeFlows(self, matrix) -> np.array:
        return np.sum(matrix, axis = 0)
    
    def prometheeI(self, preference_function: str = "Type5"):
        X = self.computeComprehensiveMatrix(preference_function)
        positive_flows = sorted([x for x in zip(self.alternatives["name"], self.computePositiveFlows(X))], key=lambda x:x[1], reverse=True)
        negative_flows = sorted([x for x in zip(self.alternatives["name"], self.computeNegativeFlows(X))], key=lambda x:x[1])
        print(*positive_flows)
        print(*negative_flows)
        # graph = []
        # pos = positive_flows[0]
        # neg = negative_flows[0]
        # for i in range(1, len(positive_flows)):
        #     next_pos = positive_flows[i]
        #     next_neg = negative_flows[i]



In [72]:
decisionMaker = Promethee(data, params)
decisionMaker.prometheeI()

[ 5  7  8  7  2  8 10  6  3]


ValueError: operands could not be broadcast together with shapes (9,22,22) (9,) 

## Electre Tri-B ##

### SRF procedure for defining weights ###

<b>The DM ranks the criteria in the following way (from the most important groups to the least important):</b>

L1: mileage <br>
<i>One white card</i> <br>
L2: fuel_consumption, price <br>
L3: power, engine_size <br>
L4: production_year <br>
L5: prestige <br>
<i>One white card</i> <br>
L6: automatic_gearbox <br>
L7: colour_preference <br>

<b><i>Importance mileage to colour_preference is Z = 4 </i></b>

In [37]:
# Mapping between criteria string names and indices
NUM_CRITERIA = 9
CRITERIA2INDEX = {
    "prestige": 0,
    "power": 1,
    "price": 2,
    "engine_size": 3,
    "colour_preference": 4,
    "fuel_consumption": 5,
    "mileage": 6,
    "production_year": 7,
    "automatic_gear_box": 8
}

# Inverse dictionary that contains indices integers as keys and string names as values
INDEX2CRITERIA = {v: k for k, v in CRITERIA2INDEX.items()}

In [12]:
# parameters to compute weights using SRF procedure
z_ratio = 4
# List that contains information about how DM ranks the criteria into groups. 
# Each list from the up to bottom contains indices of criteria assigned to a given group
# Empty lists indicate the white card.
# In below list the first nested list contains only one integer (index) which is 6 what means 
# that in the first group we have only mileage criterion.
# Next, there is an empty list which means that here we have one white card etc.
L_GROUPS = [[6],
            [],
            [2,5],
            [1,3],
            [7],
            [0],
            [],
            [8],
            [4]]

In [34]:
def compute_weights(z_ratio: int, L: list, n_criteria: int=9) -> np.array:
    # Number of groups which is the length of the list without empty lists, needed for computing r(t) - raw rank of a given group
    num_groups_left = len(L) - sum([1 for sublist in L if len(sublist) == 0])
    non_normalized_weights = np.zeros(n_criteria)
    # raw ranking of the best group
    r_v = len(L)
    for _, group in enumerate(L):
        if group:
            # r - place of the group + left white cards
            r = num_groups_left+sum([1 for sublist in L[_:] if len(sublist) == 0])
            # compute non-normalized weight for each criterion in group
            w = 1+(z_ratio-1)*(r-1)/(r_v-1)
            # put at the proper index the weight of each criterion in group
            np.put(non_normalized_weights, group, w)
            num_groups_left-=1
    return non_normalized_weights / sum(non_normalized_weights)       
    

In [43]:
WEIGHTS = compute_weights(z_ratio=20, L=L_GROUPS)
for i in range(NUM_CRITERIA):
    print(INDEX2CRITERIA[i],":", WEIGHTS[i])

prestige : 0.0818639798488665
power : 0.1297229219143577
price : 0.15365239294710328
engine_size : 0.1297229219143577
colour_preference : 0.010075566750629723
fuel_consumption : 0.15365239294710328
mileage : 0.20151133501259447
production_year : 0.10579345088161209
automatic_gear_box : 0.03400503778337532


In [92]:
# Mapping between a specific class to its name
_CLASSES = {
    1: "the worst",
    2: "bad",
    3: "better",
    4: "the best"
}

In [45]:
# boundary profiles evaluations
# each row has 9 values which indicates boundary for each criterion
BOUNDARIES = np.array([
    [2, 90, 37000, 2, 1.5, 7.1, 170000, 2000, 0], # b1 - boundary between the worst and bad class
    [5, 106, 33000, 1.6, 3.5, 6.6, 140000, 2010, 0], # b2 - boundary between bad and better class
    [8, 130, 30000, 1.2, 4.5, 5.9, 100000, 2017, 0.5] # b3 - boundary between better and the best class
])

In [57]:
b1_thresholds = np.array([
    [2, 15, 3000, 0.2, 0, 0.3, 35000, 4, 1], # p
    [1, 5, 1000, 0, 0, 0.1, 20000, 2, 0], # q
    [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, 50000, np.inf, np.inf] # veto values, only for mileage criterion
])

b2_thresholds = np.array([
    [2, 15, 3000, 0.2, 0, 0.3, 35000, 4, 1], # p
    [1, 5, 1000, 0, 0, 0.1, 20000, 2, 0], # q
    [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, 50000, np.inf, np.inf] # veto values, only for mileage criterion
])

b3_thresholds = np.array([
    [2, 15, 3000, 0.2, 0, 0.3, 35000, 4, 1], # p
    [1, 5, 1000, 0, 0, 0.1, 20000, 2, 0], # q
    [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, 50000, np.inf, np.inf] # veto values, only for mileage criterion
])

# For now all parameters (p, q, veto) are the same for all boundary profiles.

THRESHOLDS = np.array([
    b1_thresholds,
    b2_thresholds, 
    b3_thresholds
])

CRITERIA_TYPES = ["gain","gain","cost","cost","gain","cost","cost","gain","gain"]

In [54]:
ALTERNATIVES = data.loc[:, data.columns!='name'].to_numpy()

In [139]:
from typing import Tuple


class ElectreTriB:
    def __init__(self, 
                 alternatives: np.array,
                 boundaries: np.array,
                 thresholds: np.array,
                 criteria_types: list[str],
                 weights: np.array,
                 num_criteria: int = NUM_CRITERIA,
                 ) -> None:
        self.alternatives = alternatives
        self.num_alternatives = alternatives.shape[0]
        
        self.num_criteria = num_criteria
        self.boundaries = boundaries
        self.num_bound_profiles = thresholds.shape[0]
        
        self.thresholds = thresholds
        
        self.criteria_types = criteria_types
        self.weights = weights
        
    def compute_concordance_value_gain(self, a, b, p, q) -> float:
        diff = a-b
        if diff >= -q:
            return 1
        elif diff < -p:
            return 0
        else:
            return (p-(b-a))/(p-q)
            
        
    def compute_concordance_value_cost(self, a, b, p, q) -> float:
        diff = a-b
        if diff <= q:
            return 1
        elif diff > p:
            return 0
        else:
            return (p-(a-b))/(p-q)
        
        
    def compute_marginal_concordance_vectors(self, boundaries, thresholds) -> Tuple[np.array, np.array]:
        conc_matrix = np.zeros(shape=(self.num_alternatives, self.num_criteria))
        conc_matrix_inv = np.zeros(shape=(self.num_alternatives, self.num_criteria))
        
        _concordance_values_mappers = {
            "gain": self.compute_concordance_value_gain,
            "cost": self.compute_concordance_value_cost
        }
        
        for i in range(self.num_alternatives):
            for j in range(self.num_criteria):
                # read p and q value from threshold array
                p, q = thresholds[0][j], thresholds[1][j]
                g_a = self.alternatives[i][j]
                # read a boundary value of a profile for a given criterion
                g_b = boundaries[j]
                criterion_type = self.criteria_types[j]
                conc_matrix[i][j] = _concordance_values_mappers[criterion_type](
                    g_a, g_b, p, q
                )
                conc_matrix_inv[i][j] = _concordance_values_mappers[criterion_type](
                    g_b, g_a, p, q
                )
                
        # Compute the vector C(a, b_h) for all alternatives
        # this vector is one column of the final comprehensive concordance matrix
        #  C(a, b)      |  b_1  |  b_2  |  b_3
        #  -------------------------------------
        #   a1          |       |       |
        #   a2          |       |       |
        
        conc_vector = np.dot(conc_matrix, self.weights) / self.weights.sum()
        conc_vector_inv = np.dot(conc_matrix_inv, self.weights) / self.weights.sum()
                
        return (conc_vector, conc_vector_inv)
    
    def compute_discordance_value_gain(self, a: int, b: int, p: int, v: int) -> float:
        diff = a-b
        if diff <= -v:
            return 1
        elif diff >= -p:
            return 0
        else:
            return ((b-a)-p)/(v-p)
        
    def compute_discordance_value_cost(self, a: int, b: int, p: int, v: int) -> float:
        diff = a-b
        if diff >= v:
            return 1
        elif diff <= p:
            return 0
        else:
            return (v-(a-b))/(v-p)
    
    def compute_marginal_discordance_matrices(self, boundaries: np.array, thresholds: np.array) -> Tuple[np.array, np.array]:
        disc_matrix = np.zeros(shape=(self.num_alternatives, self.num_criteria))
        disc_matrix_inv = np.zeros(shape=(self.num_alternatives, self.num_criteria))
        
        _discordance_values_mappers = {
            "gain": self.compute_discordance_value_gain,
            "cost": self.compute_discordance_value_cost
        }
        
        for i in range(self.num_alternatives):
            for j in range(self.num_criteria):
                # read p and q value from threshold array
                p, v = thresholds[0][j], thresholds[2][j]
                g_a = self.alternatives[i][j]
                # read a boundary value of a profile for a given criterion
                g_b = boundaries[j]
                criterion_type = self.criteria_types[j]
                disc_matrix[i][j] = _discordance_values_mappers[criterion_type](
                    g_a, g_b, p, v
                )
                disc_matrix_inv[i][j] = _discordance_values_mappers[criterion_type](
                    g_b, g_a, p, v
                )
                
        return (np.nan_to_num(disc_matrix), np.nan_to_num(disc_matrix_inv))
    
    
    def compute_credibility_vectors(self,
                                      conc_vector: np.array,
                                      conc_vector_inv: np.array,
                                      disc_matrix: np.array,
                                      disc_matrix_inv: np.array) -> Tuple[np.array, np.array]:
        credibility = np.zeros(shape=(self.num_alternatives))
        credibility_inv = np.zeros(shape=(self.num_alternatives))
        
        for i in range(self.num_alternatives):
            credit_value = conc_vector[i]
            credit_inv_value = conc_vector_inv[i]
            for j in range(self.num_criteria):
                if disc_matrix[i][j] > conc_vector[i]:
                    credit_value *= (1-disc_matrix[i][j])/(1-conc_vector[i])
                if disc_matrix_inv[i][j] > conc_vector_inv[i]:
                    credit_inv_value += (1-disc_matrix_inv[i][j])/(1-conc_vector_inv[i])
            credibility[i] = credit_value
            credibility_inv[i] = credit_inv_value
        
        return (credibility, credibility_inv)
    
    def compute_relations(self, 
                          credibility: np.array,
                          credibility_inv: np.array,
                          lambda_cut: float = 0.7) -> list[str]:
        relations = []
        for i in range(self.num_alternatives):
            if credibility[i] >= lambda_cut and credibility_inv[i] < lambda_cut:
                relations.append(">")
            elif credibility[i] < lambda_cut and credibility_inv[i] >= lambda_cut:
                relations.append("<")
            elif credibility[i] >= lambda_cut and credibility_inv[i] >= lambda_cut:
                relations.append("I")
            elif credibility[i] < lambda_cut and credibility_inv[i] < lambda_cut:
                relations.append("?")
        return relations
        
    
    def make_classification_optimistic(self, relations: list[str], classes: dict[int, str]):
        for i, e in enumerate(relations):
            if e == "<":
                return classes[i+1]
        return classes[4]
            
        
    def make_classification_pessimistic(self, relations: list[str], classes: dict[int, str]):
        for i, e in reversed(list(enumerate(relations))):
            if e == "I" or e == ">":
                return classes[i+2]
        return classes[1]
        
    def classify_alternatives(self, alternatives_names: list[str], classes: dict[int, str], lambda_cut: float = 0.7):
        data_results = {"name": alternatives_names}
        
        results = pd.DataFrame(data_results)
        
        for i in range(self.num_bound_profiles):
            boundaries = self.boundaries[i]
            thresholds = self.thresholds[i]
            
            conc_vector, conc_vector_inv = self.compute_marginal_concordance_vectors(boundaries, thresholds)
            disc_matrix, disc_matrix_inv = self.compute_marginal_discordance_matrices(boundaries, thresholds)
            
            credibility, credibility_inv = self.compute_credibility_vectors(conc_vector,
                                                                            conc_vector_inv, 
                                                                            disc_matrix,
                                                                            disc_matrix_inv)
            
            relations = self.compute_relations(credibility, credibility_inv, lambda_cut)
            results["b"+str(i+1)] = relations
        # results = results.reset_index()
        pessimistic = []
        optimistic = []
        for _, row in results.iterrows():
            relations = [row.b1, row.b2, row.b3]
            pessimistic_class = self.make_classification_pessimistic(relations, classes)
            pessimistic.append(pessimistic_class)
            optimistic_class = self.make_classification_optimistic(relations, classes)
            optimistic.append(optimistic_class)
        results["PES(a)"] = pessimistic
        results["OPT(a)"] = optimistic
            
            
            
        return results
    

In [140]:
test = ElectreTriB(
    alternatives=ALTERNATIVES,
    boundaries=BOUNDARIES,
    thresholds=THRESHOLDS,
    criteria_types=CRITERIA_TYPES,
    weights=WEIGHTS
    
)

In [141]:
alternatives_names = data['name'].to_list()

In [142]:
test.classify_alternatives(alternatives_names, lambda_cut=0.7, classes=_CLASSES)

C:\Users\janko\AppData\Local\Temp\ipykernel_57748\3561420967.py:97: RuntimeWarning: invalid value encountered in double_scalars
  return (v-(a-b))/(v-p)
C:\Users\janko\AppData\Local\Temp\ipykernel_57748\3561420967.py:97: RuntimeWarning: invalid value encountered in double_scalars
  return (v-(a-b))/(v-p)
C:\Users\janko\AppData\Local\Temp\ipykernel_57748\3561420967.py:97: RuntimeWarning: invalid value encountered in double_scalars
  return (v-(a-b))/(v-p)


,name,b1,b2,b3,PES(a),OPT(a)
0,Seat Ibiza 1.2 white,>,>,<,better,better
1,Seat Ibiza 1.4 green,>,?,<,bad,better
2,Skoda Fabia grey,?,?,<,the worst,better
3,Nissan Note grey,>,>,<,better,better
4,MINI Cooper 1.6 blue,>,?,?,bad,the best
5,Opel Corsa white,>,>,<,better,better
6,Toyota Yaris 1.3 grey,I,I,<,better,better
7,Citroen DS3 red,>,>,<,better,better
8,Volkswagen Polo 1.2 white,>,?,<,bad,better
9,Toyota Yaris 1.0 white,>,?,<,bad,better


## Data set ##

1. What is the domain of the problem about?

    <i>The domain of the problem is set of different cars available on the polish market</i>

2. What is the source of the data?

    <i>The data is taken from the website https://www.otomoto.pl/, where people or companies around Poland are selling their cars.</i>

3. What is the point of view of the decision maker?

    <i>The decision maker is looking for a car suitable for the city, therefore the finally choosen car should be not only powerful, but also economic.</i>

4. What is the number of alternatives considered? Were there more of them in the original data set?

    <i>We've gathered data about 22 cars. As a original dataset we may consider the website https://www.otomoto.pl/. For our search filters there were available more than 3000 cars.</i>

5. Describe one of the alternatives considered (give its name, evaluations, specify preferences for this
alternative)

    <i>Considering the first alternative from our dataset it is a Seat from 2015 year. It has quite small engine (1.2) and fuel consumption (6.0) what is good, because we are looking for the car reasonable for the city (economic). However, the power of the engine is not as good as it could be, it is only 90HP. The price is 34900pln what is the upper boundary looking through the entire dataset. TODO</i>